In [16]:
import os
import nibabel as nib
from monai.metrics import DiceMetric, HausdorffDistanceMetric, ConfusionMatrixMetric
import torch
import matplotlib.pyplot as plt



In [29]:

pred_folder = "/home/linuxlia/Lia_Masterthesis/phuse_thesis_2024/thesis_experiments/01_aschoplex_from_scratch/working_directory_T1_240820_1823/ensemble_output/image_Ts"
gt_folder = "/home/linuxlia/Lia_Masterthesis/data/reference_labels/ref_labelTs"

pred_files = sorted([f for f in os.listdir(pred_folder) if f.endswith('.nii.gz')])
gt_files = sorted([f for f in os.listdir(gt_folder) if f.endswith('.nii')])

predictions = [nib.load(os.path.join(pred_folder, f)).get_fdata() for f in pred_files]
ground_truths = [nib.load(os.path.join(gt_folder, f)).get_fdata() for f in gt_files]

# Initialize the metric
dice_metric = DiceMetric(include_background=False, reduction="mean")
hausdorff_metric = HausdorffDistanceMetric(include_background=False, percentile=95)
confusion_matrix_metric = ConfusionMatrixMetric(include_background=False, metric_name=["precision", "recall", "f1_score"], reduction="mean")


dice_scores = []
hd_distances = []
f1_scores = []
precisions = []
recalls = []

# Compute metrics for each pair of prediction and ground truth
for pred, gt in zip(predictions, ground_truths):
    pred_tensor = torch.tensor(pred).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimension
    gt_tensor = torch.tensor(gt).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimension
    
    dice_score = dice_metric(y_pred=pred_tensor, y=gt_tensor)
    #print("dice_metric", dice_score)
    mean_dice_score = dice_score.mean().item()
    dice_scores.append(mean_dice_score)

    # Compute Hausdorff distance
    hd_distance = hausdorff_metric(y_pred=pred_tensor, y=gt_tensor)
    #print("hausdorff_metric", hd_distance)
    hd_distances.append(hd_distance.item())
    

    # Compute Precision, Recall, and F1 Score
    confusion_matrix_metric(y_pred=pred_tensor, y=gt_tensor)
    precision, recall, f1_score = confusion_matrix_metric.aggregate()

    #print(precision, recall, f1_score)
    f1_scores.append(f1_score)
    precisions.append(precision)
    recalls.append(recall)


mean_dice = sum(dice_scores) / len(dice_scores)
mean_hd_distance = sum(hd_distances) / len(hd_distances)
precision = sum(precisions) / len(precisions)
recall = sum(recalls) / len(recalls)
f1_score = sum(f1_scores) / len(f1_scores)

print(f"Mean Dice: {mean_dice}")
print(f"Mean Hausdorff Distance: {mean_hd_distance}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")





Mean Dice: 0.8194468529136093
Mean Hausdorff Distance: 2.2686533442250005
Precision: tensor([0.8804])
Recall: tensor([0.7432])
F1 Score: tensor([0.8051])


In [31]:
import os
import nibabel as nib
import torch
from monai.metrics import DiceMetric, HausdorffDistanceMetric, ConfusionMatrixMetric
import pandas as pd

# List of prediction folders
pred_folders = [
    "/home/linuxlia/Lia_Masterthesis/phuse_thesis_2024/thesis_experiments/01_aschoplex_from_scratch/working_directory_T1_240820_1823/ensemble_output/image_Ts",
    "/home/linuxlia/Lia_Masterthesis/phuse_thesis_2024/thesis_experiments/01_aschoplex_from_scratch/working_directory_T1xFLAIR_240822/ensemble_output/image_Ts",
    "/home/linuxlia/Lia_Masterthesis/phuse_thesis_2024/thesis_experiments/01_aschoplex_from_scratch/working_directory_FLAIR_240823/ensemble_output/image_Ts",
]

gt_folder = "/home/linuxlia/Lia_Masterthesis/data/reference_labels/ref_labelTs"
gt_files = sorted([f for f in os.listdir(gt_folder) if f.endswith('.nii')])
ground_truths = [nib.load(os.path.join(gt_folder, f)).get_fdata() for f in gt_files]

# Initialize the metrics
dice_metric = DiceMetric(include_background=False, reduction="mean")
hausdorff_metric = HausdorffDistanceMetric(include_background=False, percentile=95)
confusion_matrix_metric = ConfusionMatrixMetric(include_background=False, metric_name=["precision", "recall", "f1_score"], reduction="mean")

# List to store the results
results_list = []

# Loop over each prediction folder
for pred_folder in pred_folders:
    pred_files = sorted([f for f in os.listdir(pred_folder) if f.endswith('.nii.gz')])
    predictions = [nib.load(os.path.join(pred_folder, f)).get_fdata() for f in pred_files]

    dice_scores = []
    hd_distances = []
    f1_scores = []
    precisions = []
    recalls = []

    # Compute metrics for each pair of prediction and ground truth
    for pred, gt in zip(predictions, ground_truths):
        pred_tensor = torch.tensor(pred, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
        gt_tensor = torch.tensor(gt, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        
        # Check for empty tensors
        if torch.sum(pred_tensor) == 0 or torch.sum(gt_tensor) == 0:
            print("Warning: Empty prediction or ground truth tensor detected.")
            continue
        
        # Ensure matching shapes
        if pred_tensor.shape != gt_tensor.shape:
            print(f"Error: Shape mismatch - pred: {pred_tensor.shape}, gt: {gt_tensor.shape}")
            continue
        
        # Compute Dice score
        dice_score = dice_metric(y_pred=pred_tensor, y=gt_tensor)
        mean_dice_score = dice_score.mean().item()
        dice_scores.append(mean_dice_score)
        
        # Compute Hausdorff distance
        hd_distance = hausdorff_metric(y_pred=pred_tensor, y=gt_tensor)
        hd_distances.append(hd_distance.item())
        
        # Accumulate confusion matrix results
        confusion_matrix_metric(y_pred=pred_tensor, y=gt_tensor)
        precision, recall, f1_score = confusion_matrix_metric.aggregate()
        
        f1_scores.append(f1_score.item())
        precisions.append(precision.item())
        recalls.append(recall.item())

    # Aggregate metrics to get summary statistics
    mean_dice = sum(dice_scores) / len(dice_scores)
    mean_hd_distance = sum(hd_distances) / len(hd_distances)
    mean_precision = sum(precisions) / len(precisions)
    mean_recall = sum(recalls) / len(recalls)
    mean_f1_score = sum(f1_scores) / len(f1_scores)

    # Append the results to the list
    results_list.append({
        "Model": os.path.basename(pred_folder),
        "Mean Dice": mean_dice,
        "Mean Hausdorff Distance": mean_hd_distance,
        "Precision": mean_precision,
        "Recall": mean_recall,
        "F1 Score": mean_f1_score
    })

# Convert the results list to a DataFrame
results = pd.DataFrame(results_list)

# Print the results table
print(results)

      Model  Mean Dice  Mean Hausdorff Distance  Precision    Recall  F1 Score
0  image_Ts   0.819447                 2.268653   0.880423  0.743209  0.805111
1  image_Ts   0.874529                 1.494537   0.895855  0.784857  0.836601
2  image_Ts   0.822920                 2.095111   0.874690  0.816498  0.844546


In [ ]:

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Prediction")
plt.imshow(predictions[0][..., int(predictions[0].shape[-1] / 2)], cmap="gray")

plt.subplot(1, 2, 2)
plt.title("Ground Truth")
plt.imshow(ground_truths[0][..., int(ground_truths[0].shape[-1] / 2)], cmap="gray")

plt.show()
